In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm
import seaborn as sns
pd.set_option('display.max_columns',None)
from sklearn.metrics import roc_auc_score
import pickle
TARGET = 'answered_correctly'

In [3]:
# with open('../models/101_simple_lgb.pkl','rb') as f:
#     model1 = pickle.load(f)
pred1 = pd.read_feather('../data/oof/101_simple_lgb.feather')
pred2 = pd.read_feather('../data/oof/103_last_lgb.feather')
pred3 = pd.read_feather('../data/oof/104_last_lgb.feather')

In [4]:
df = pd.read_feather('../features/all_data/BASE_FIX_valid.feather')

In [5]:
pred1.columns = ['row_id','pred1']
pred2.columns = ['row_id','pred2']
pred3.columns = ['row_id','pred3']
df = df[[TARGET]]

df = pd.concat([df,pred1['pred1'],pred2['pred2'],pred3['pred3']],axis=1)

In [7]:
roc_auc_score(df[TARGET],df['pred1']*2)

0.7906350845180423

In [21]:
roc_auc_score(df[TARGET],df['pred2']*2)

0.7904026782773977

In [8]:
roc_auc_score(df[TARGET],df['pred3']*2)

0.7905755630346007

In [22]:
roc_auc_score(df[TARGET],df['pred1']*1.1+df['pred3'])

0.791016440421306

In [25]:
roc_auc_score(df[TARGET],df['pred1']*1.1+df['pred3'])

0.791016440421306

In [ ]:
roc_auc_score(df[TARGET],df['pred1']+df['pred2']*0.8)

In [5]:
USE_COLS = ['prior_question_elapsed_time', 'prior_question_had_explanation', 'part','timestamp',
            'ans_user_avg', 'ans_user_count','elapsed_time_user_avg', 'explanation_user_avg','elapsed_time_user_sum',
            'ans_content_avg', 'elapsed_time_content_avg', 'explanation_content_avg',
            'user_content_count',
            'lag_time_1', 'lag_time_2', 'lag_time_3','lag_incorrect_time',
            'lag_part_time_1','lag_part_time_2','lag_part_time_3',
            "tags_pca_0", "tags_pca_1",'tags1','tags2','tags3','tags_nan_count','tags_nan_count_mean','tags1_cut_mean','tags1_cut',
            'first_bundle',
            'ans_part_mean','ans_user_part_avg','user_part_count',
            'ans_community_mean','community',
            'content_id','bundle_id',
            'first_bundle_cut','first_bundle_cut_mean',
            'rolling_mean_10','rolling_mean_3','rolling_part_mean_10','rolling_part_mean_3',
            'tags_mean','tags_elapsed_time_avg','tags_explanation_avg']

In [6]:
def load_features(features_list,path,train_valid):

    dfs = [pd.read_feather(f'../{path}/{feature}_{train_valid}.feather') for feature in features_list]
    df = pd.concat(dfs,axis=1)
    return df

In [7]:
path = 'all_data'
FEATURES_LIST = ['BASE_FIX','LOOP_FIX_TIME5','TAGS','GROUP_BY','BUNDLE_ID','ROLLING_MEAN2','ROLLING_PART_MEAN3','CONTENT_ID','TAGS_MEAN']
valid = load_features(FEATURES_LIST,path=f'features/{path}',train_valid='valid')

In [ ]:
valid['pred1'] = model1.predict(valid[USE_COLS])
valid['pred2'] = model2.predict(valid[USE_COLS])

In [14]:
roc_auc_score(valid[TARGET],valid['pred1'])

0.7848071122915671

In [15]:
roc_auc_score(valid[TARGET],valid['pred2'])

0.7846245795206664

In [21]:
roc_auc_score(valid[TARGET],valid['pred1']+valid['pred3']*10)

0.7889486913660605

In [2]:
val = pd.read_feather('../features/kernel_base/BASE_FIX_valid.feather')
oof1 = pd.read_feather('../data/oof/064_simple_lgb.feather')
oof2 = pd.read_feather('../data/oof/065_simple_lgb.feather')
# oof3 = pd.read_feather('../data/oof/060_simple_lgb.feather')

In [3]:
oof1.columns = ['row_id','pred1']
oof2.columns = ['row_id','pred2']
# oof3.columns = ['row_id','pred3']

In [4]:
df = pd.concat([val,oof1['pred1'],oof2['pred2']],axis=1)

In [14]:
df['pred'] = df['pred1'] + df['pred2']*1.5

In [15]:
roc_auc_score(df[TARGET],df['pred'])

0.7843040247864422